In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget
%pdb off

from pyCascade import probePost, physics, utils, probeReadWrite
from pyCascade.probeReadWrite import read_probes_file_switch
from filloutVentilationStats import *
from matplotlib import pyplot as plt
from matplotlib import cm, colors
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors
import numpy as np
import scipy as sp
import os
from IPython.core.debugger import set_trace
import pandas as pd
import seaborn as sns
from cycler import cycler
import plotly.express as px
import plotly
import plotly.graph_objects as go
from plotly.offline import plot
from plotly.subplots import make_subplots
from IPython.display import display, HTML
import statsmodels.api as sm
import warnings
import ast
from scipy.optimize import minimize
import seaborn as sns

plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

plt.rcParams['figure.dpi'] = 140
im_scaling = .75
plt.rcParams['figure.figsize'] = [6.4 * im_scaling, 4.8 * im_scaling]

############ Universal ################
scratch_home = os.getenv('SCRATCH') #need to set SCRATCH (even if there is no real SCRATCH) to the location where results are written
scratch_dir = f'{scratch_home}/Cascade/city_block_cfd'
home_dir = !pwd
home_dir = home_dir[0]

display(scratch_dir)
display(home_dir)
plt.close('all')

# Setup

## Runs

In [2]:
multiRun_dir = f"{home_dir}/CHARLES/multiRuns/"
plotFolder = f"{multiRun_dir}"

## Read in results

In [3]:
def evalStringAsList(s):
    if isinstance(s, str) and s[0] == "[" and s[-1] == "]":
        return ast.literal_eval(s)
    else:
        return s

In [4]:
flowStatsMI = pd.read_csv(f"{multiRun_dir}/flowStatsMI.csv", index_col = [0,1])
roomVentilationMI = pd.read_csv(f"{multiRun_dir}/roomVentilationMI.csv", index_col = [0,1])


In [5]:
# flowStatsMI = flowStatsMI.loc[[
#     2530, 2531, 2532, 
#     2540, 2541, 2542, 
#     2550, 2551, 2552, 
#     2560, 2561, 2562, 
#     3210, 3211, 3212, 
#     3220, 3221, 3222, 
#     3230, 3231, 3232, 
#     3240, 3241, 3242, 
# ]]
# roomVentilationMI = roomVentilationMI.loc[[
#     2530, 2531, 2532, 
#     2540, 2541, 2542, 
#     2550, 2551, 2552, 
#     2560, 2561, 2562, 
#     3210, 3211, 3212, 
#     3220, 3221, 3222, 
#     3230, 3231, 3232, 
#     3240, 3241, 3242, 
# ]]

In [6]:
# flowStatsMI = flowStatsMI.loc[[
#     2460, 2461, 2462, 
#     2470, 2471, 2472, 
#     2480, 2481, 2482, 
#     2490, 2491, 2492, 
#     3160, 3161, 3162, 
#     3170, 3171, 3172, 
#     3180, 3181, 3182, 
#     3190, 3191, 3192, 
# ]]
# roomVentilationMI = roomVentilationMI.loc[[
#     2460, 2461, 2462, 
#     2470, 2471, 2472, 
#     2480, 2481, 2482, 
#     2490, 2491, 2492, 
#     3160, 3161, 3162, 
#     3170, 3171, 3172, 
#     3180, 3181, 3182, 
#     3190, 3191, 3192, 
# ]]

In [7]:
# flowStatsMI = combine_stats(flowStatsMI, ["WS", "delT", "SS", "C", "A", "slAll"], index_col = "csId")
# roomVentilationMI = combine_stats(roomVentilationMI,   ["WS", "delT", "SS", "C", "A", "slAll"], index_col = "csId")

# flowStatsMI = flowStatsMI[flowStatsMI["slAll"] == True]
# roomVentilationMI = roomVentilationMI[roomVentilationMI["slAll"] == True]

In [8]:
# 2*flowStatsMI2.loc[2495]["rms-sn_prod(abs(u))-Norm"]**2 - flowStatsMI.loc[2460]["rms-sn_prod(abs(u))-Norm"]**2 - flowStatsMI.loc[2530]["rms-sn_prod(abs(u))-Norm"]**2

In [9]:
runsNoInt = {
    257: {'A': 45, 'WS': 2, 'C': 2},
    258: {'A': 45, 'WS': 4, 'C': 2},
    259: {'A': 0,  'WS': 2, 'C': 2},
    260: {'A': 0,  'WS': 4, 'C': 2},

    325: {'A': 45, 'WS': 2, 'C': 3},
    326: {'A': 45, 'WS': 4, 'C': 3},
    327: {'A': 0,  'WS': 2, 'C': 3},
    328: {'A': 0,  'WS': 4, 'C': 3},
}

In [ ]:
pcOverwrite = True

def getNoIntRun(df, A, WS, C):
    filtered = df[(df['A'] == A) & (df['WS'] == WS) & (df['C'] == C)]
    if not filtered.empty:
        return filtered.index[0]
    return None

if pcOverwrite or os.path.exists(f"{multiRun_dir}/pointCloudStatsNoIntMI.csv") == False:
    # pointCloud pressure probes
    runIndex = flowStatsMI.index.get_level_values(0).unique()
    runIndex = np.sort(runIndex)
    pcStats = pd.DataFrame()

    def addB(name):
        if name.split("_")[-1][0] != "B":
            return f"{name}_B"
        return name

    for run in runIndex:
        A = flowStatsMI.loc[run, "A"].iloc[0]
        WS = flowStatsMI.loc[run, "WS"].iloc[0]
        C = flowStatsMI.loc[run, "C"].iloc[0]
        R = getNoIntRun(pd.DataFrame(runsNoInt).T, A, WS, C)
        V = run % 10
        if V == 0:
            oak_probes_dir =  f'{home_dir}/CHARLES/config{C}/R{R%100}/probes/probesOut_parquet/pointCloud'
            print(oak_probes_dir)
            probes = probePost.Probes(
                oak_probes_dir, 
                probe_type = "POINTCLOUD_PROBES", 
                file_type = "parquet", 
                directory_parquet = oak_probes_dir,
            )
            stats = probes.statistics()
            stats = stats.map(lambda df: df["p_avg"].mean())

            stats.index = stats.index.map(addB)
            stats = stats.rename(columns={119000: 'pNoInt'})
        stats["run"] = run
        stats["runNoInt"] = R * 10 + V
        pcStats = pd.concat([pcStats, stats], axis = "rows")


        probes_dir = f'{home_dir}/CHARLES/config{C}/R{R%100}/probes/probesOut_parquet'
        EPprobes = probePost.Probes(probes_dir, directory_parquet = probes_dir, file_type = "parquet", name_pattern = "extraProbe")

        nameKey = read_probes_file_switch(f"{probes_dir}/../locations/nameKey_extraProbe.txt")
        nameKey = nameKey.compute()
        nameKey = pd.concat([nameKey, EPprobes.locations["extraProbe"]], axis = "columns")

        df = EPprobes.statistics(
            names = [name for name in  EPprobes.probe_names if "extraProbe" in name], 
            steps = [119000],
            quants = ["comp(u_avg,0)", "comp(u_avg,1)", "comp(u_avg,2)", "mag(u)_avg", "p_avg", "D_avg", "S_avg", "T_avg"],
            parrallel=False
        )

        extraProbe = nameKey.copy()
        extraProbe = pd.concat([extraProbe, df], axis = "columns")
        extraProbe.set_index(0, inplace=True)
        extraProbe = extraProbe.rename(columns=lambda x: f"EP_{x}")
        extraProbe = extraProbe.rename(index=lambda x: x.replace("extraProbe_", ''))

        

    pcStatsMI = pcStats.reset_index()
    pcStatsMI = pcStatsMI.set_index(['run', 'index'])
    pcStatsMI.to_csv(f"{multiRun_dir}/pointCloudStatsNoIntMI.csv")
else:
    pcStatsMI = pd.read_csv(f"{multiRun_dir}/pointCloudStatsNoIntMI.csv", index_col=[0,1])

In [ ]:
extraProbe

In [ ]:
flowStatsMI["pNoInt"] = np.nan
flowStatsMI["runNoInt"] = np.nan
flowStatsMI.update(pcStatsMI)
flowStatsMI

In [ ]:
g = 10
beta = 0.0034
rho = 1.225
hm = 6
window_dim = hm/2/4
A = window_dim ** 2

def getWindBuoyantP(rho, flowParams):
    p_w = flowParams["p_w"]
    z = flowParams["z"]
    delT = flowParams["delT"]
    delrho = -rho * beta * delT
    return (delrho * g * z) + p_w # delP is outdoor minus indoor, while p0/rho is indoor minus outdoor, driving positive flow into the room (oppiste textbook)

def flowFromP(rho, C_d, A, delp):
    delp=np.array(delp)
    S = np.zeros_like(delp)
    S[delp!=0] = abs(delp[delp!=0]) / delp[delp!=0]
    return S * C_d * A * np.sqrt(2 * abs(delp) / rho)

def CFromFlow(rho, q, A, delp):
    delp=np.array(delp)
    S = np.zeros_like(delp)
    S[delp!=0] = abs(delp[delp!=0]) / delp[delp!=0]
    C = q /  (S * A * np.sqrt(2 * abs(delp) / rho))
    return C

def flowField(p_0, rho, flowParams):
    C_d = flowParams["C_d"]
    A = flowParams["A"]
    rooms = flowParams["rooms"]
    delP = -np.matmul(rooms, p_0) + getWindBuoyantP(rho, flowParams) 
    return flowFromP(rho, C_d, A, delP)

def getC(p_0, rho, flowParams):
    A = flowParams["A"]
    q = flowParams["q"]
    rooms = flowParams["rooms"]
    delP = -np.matmul(rooms, p_0) + getWindBuoyantP(rho, flowParams)
    return CFromFlow(rho, q, A, delP)

def qObjective(p_0, rho, flowParams):
    qs = flowField(p_0, rho, flowParams)
    rooms = flowParams["rooms"]
    qRooms = np.matmul(rooms.T, qs)
    return np.sum(qRooms**2)

def findOptimalP0(rho, flowParams):
    bounds = np.array([np.min(getWindBuoyantP(rho, flowParams)), np.max(getWindBuoyantP(rho, flowParams))])
    x0 = np.mean(bounds)
    NRooms = flowParams["rooms"].shape[1]
    bounds = np.tile(bounds, (NRooms, 1))
    x0 = np.tile(x0, NRooms)
    return minimize(qObjective, x0=x0, bounds=bounds, args=(rho, flowParams))

In [ ]:
flowParams = {
    "C_d": [1],
    "A": [1],
    "p_w": [0],
    "z": [3],
    "delT": [3],
    "q": [1],
    "rooms": [[1]]
}

flowParams = utils.dict_apply(np.array)(flowParams)

p_0s = np.array([0])

display(flowField(p_0s, rho, flowParams))
display(qObjective(p_0s, rho, flowParams))
display(getC(p_0s, rho, flowParams))

In [ ]:
flowParams = {
    "C_d": [1, 1, 1, 1],
    "A": [1, 1, 1, 1],
    "p_w": [1, 3, 1, -3],
    "z": [3, 3, 6, 3],
    "delT": [-3, 0, 3, 0],
    "q": [2, 2, -1, -3],
    "rooms": [[1], [1], [1], [1]]
}

flowParams = utils.dict_apply(np.array)(flowParams)

p_0s = np.array([1])

display(flowField(p_0s, rho, flowParams))
display(qObjective(p_0s, rho, flowParams))
display(getC(p_0s, rho, flowParams))

In [ ]:
optResults = findOptimalP0(rho, flowParams)
p_0s = optResults.x

display(optResults)
display(flowField(p_0s, rho, flowParams))
display(qObjective(p_0s, rho, flowParams))
display(getC(p_0s, rho, flowParams))

In [ ]:
plt.figure()
p0s = np.linspace(min(getWindBuoyantP(rho, flowParams)), max(getWindBuoyantP(rho, flowParams)), 100)
qs = [qObjective(np.array([p0]), rho, flowParams) for p0 in p0s]
plt.plot(p0s, qs)
plt.xlabel("p0")
plt.ylabel("$\sum q^2$")

In [ ]:
flowParams = {
    "C_d": [1, 1, 1, 1, 10],
    "A": [1, 1, 1, 1, 3],
    "p_w": [1, 3, 1, -3, 0],
    "z": [3, 3, 6, 3, 0],
    "delT": [-3, 0, 3, 0, 0],
    "q": [2, 2, -1, -3, -4],
    "rooms": [[1, 0], [1, 0], [0, 1], [0, 1], [1, -1]]
}

flowParams = utils.dict_apply(np.array)(flowParams)
p_0s = np.array([1, -1])

display(flowField(p_0s, rho, flowParams))
display(qObjective(p_0s, rho, flowParams))
display(getC(p_0s, rho, flowParams))

In [ ]:
optResults = findOptimalP0(rho, flowParams)
p_0s = optResults.x

display(optResults)
display(flowField(p_0s, rho, flowParams))
display(qObjective(p_0s, rho, flowParams))
display(getC(p_0s, rho, flowParams))


In [ ]:
from scipy.optimize import minimize

def getC_ds_AofA(aCorner, aCross, aDual, aSingle, aSinA, aCosA, AofA = 180, roomType="corner"):
    AofA *= np.pi / 180
    C_d = aSinA * np.sin(AofA) + aCosA * np.cos(AofA)
    if roomType == "corner":
        return C_d + aCorner
    if roomType == "cross":
        return C_d + aCross
    if roomType == "dual":
        return C_d + aDual
    if roomType == "single":
        return C_d + aSingle
    
def getC_ds_AofA2(aCorner, aCross, aSingle, aCornerSinA, aCrossSinA, AofA = 180, roomType="corner"):
    AofA *= np.pi / 180
    if roomType == "corner":
        return aCornerSinA * np.sin(AofA) + aCorner
    if roomType == "cross":
        return aCrossSinA * np.sin(AofA) + aCross
    if roomType == "dual":
        return aCornerSinA * np.sin(AofA) + aCorner
    if roomType == "single":
        return aSingle

def getC_ds_AofA3(aCorner, aCross, aDual, aSingle, aSinA, aCosA, sSin2a, aCos2A, AofA = 180, roomType="corner"):
    AofA *= np.pi / 180
    C_d = aSinA * np.sin(AofA) + aCosA * np.cos(AofA) + sSin2a * np.sin(2 * AofA) + aCos2A * np.cos(2 * AofA)
    if roomType == "corner":
        return C_d + aCorner
    if roomType == "cross":
        return C_d + aCross
    if roomType == "dual":
        return C_d + aDual
    if roomType == "single":
        return C_d + aSingle

def getC_ds_EP(aCorner, aCross, aDual, aSingle, aShear, aNormal, shear=0, normal=0, roomType="corner"):
    C_d = aShear * shear + aNormal * normal
    if roomType == "corner":
        return C_d + aCorner
    if roomType == "cross":
        return C_d + aCross
    if roomType == "dual":
        return C_d + aDual
    if roomType == "single":
        return C_d + aSingle

def getC_ds_All(aCorner, aCross, aDual, aSingle, aSinA, aCosA,  aShear, aNormal, aWS, AofA = 180, shear=0, normal=0, WS=0, roomType="corner"):
    AofA *= np.pi / 180
    C_d = aSinA * np.sin(AofA) + aCosA * np.cos(AofA) + aShear * shear + aNormal * normal + aWS * WS
    if roomType == "corner":
        return C_d + aCorner
    if roomType == "cross":
        return C_d + aCross
    if roomType == "dual":
        return C_d + aDual
    if roomType == "single":
        return C_d + aSingle

def getC_ds_opening(aX, aZ, aXSinA, aZSinA, aXCosA, aZCosA, AofA=0, openingType="xwindow"):
    AofA *= np.pi / 180
    if openingType == "xwindow":
        return aX + aXSinA * np.sin(AofA) + aXCosA * np.cos(AofA)
    if openingType == "zwindow":
        return aZ + aZSinA * np.sin(AofA) + aZCosA * np.cos(AofA)

def getC_ds(params, typeC_d="AofA", AofA=0, shear=0, normal=0, openingType="xwindow", roomType="corner"):
    if typeC_d == "AofA":
        return getC_ds_AofA(*params, AofA=AofA, roomType=roomType)
    if typeC_d == "AofA2":
        return getC_ds_AofA2(*params, AofA=AofA, roomType=roomType)
    if typeC_d == "AofA3":
        return getC_ds_AofA3(*params, AofA=AofA, roomType=roomType)
    if typeC_d == "EP":
        return getC_ds_EP(*params, shear=shear, normal=normal, roomType=roomType)
    if typeC_d == "All":
        return getC_ds_All(*params, AofA=AofA, shear=shear, normal=normal, WS=1, roomType=roomType)
    if typeC_d == "opening":
        return getC_ds_opening(*params, openingType=openingType)



def update_flow_and_ventilation(flowStatsMI, roomVentilationMI, paramsC_d, useDoors=True, pTypes = {"pNoInt": "pNoInt"}, typeC_d="AofA"):
    flowStatsMI["cosAofA"] = np.round(np.cos(flowStatsMI["AofA"] * np.pi / 180), 2)
    flowStatsMI["sinAofA"] = np.round(np.sin(flowStatsMI["AofA"] * np.pi / 180), 2)

    for pType in pTypes:
        roomVentilationMI[f"{pType}-success"] = False
    for (run, room), row in roomVentilationMI.iterrows():
        flowParams = {
            "C_d": [],
            "A": [],
            "z": [],
            "delT": [],
            "q": [],
            "rooms": [],
        }
        windowKeyCols = roomVentilationMI.columns[
            roomVentilationMI.columns.str.contains("windowKeys")
        ].tolist()
        windowKeys = row[windowKeyCols].dropna()

        for pType in pTypes:
            flowParams[pType] = []
        for windowKey in windowKeys:
            for pType, pCol in pTypes.items():
                flowParams[pType].append(flowStatsMI.loc[(run, windowKey), pCol])
            C_d = getC_ds(
                paramsC_d,
                typeC_d = typeC_d,
                AofA=flowStatsMI.loc[(run, windowKey),"AofA"], 
                shear=flowStatsMI.loc[(run, windowKey),"EP_shear"], 
                normal=flowStatsMI.loc[(run, windowKey),"EP_normal"],
                openingType = flowStatsMI.loc[(run, windowKey),"openingType"],
                roomType = row["roomType"],
                )
            flowParams["C_d"].append(C_d)
            flowParams["A"].append(A)
            flowParams["z"].append(flowStatsMI.loc[(run, windowKey), "y"])  # y is vertical in simulation
            flowParams["delT"].append(row["mean-T-room"])
            flowParams["q"].append(flowStatsMI.loc[(run, windowKey), "mean-mass_flux"])
            if "dual" in room and useDoors:
                roomCord = windowKey.split("_")[1]
                if roomCord == "0-1":
                    roomRow = [1, 0]
                elif roomCord == "1-1":
                    roomRow = [0, 1]
                else:
                    raise Exception(f"Unrecognized room {roomCord} in dual room")
            else:
                roomRow = [1]
            flowParams["rooms"].append(roomRow)

        if "dual" in room and useDoors:
            H = 3
            for pType in pTypes:
                flowParams[pType].append(0)
            flowParams["C_d"].append(1)
            flowParams["A"].append(A * 3)
            flowParams["z"].append(H / 2)
            flowParams["delT"].append(row["mean-T-room"])
            qRooms = np.matmul(np.array(flowParams["rooms"]).T, np.array(flowParams["q"]))
            flowParams["q"].append(np.diff(qRooms)[0])
            flowParams["rooms"].append([1, -1])

        flowParams = utils.dict_apply(np.array)(flowParams)

        sinAofAs = []
        cosAofAs = []
        for i, windowKey in enumerate(windowKeys):
            sinAofAs.append(flowStatsMI.loc[(run, windowKey), "sinAofA"])
            cosAofAs.append(flowStatsMI.loc[(run, windowKey), "cosAofA"])

        roomVentilationMI.loc[(run, room), "sinAofA"] = np.mean(sinAofAs)
        roomVentilationMI.loc[(run, room), "cosAofA"] = np.mean(cosAofAs)

        for pType in pTypes:
            NRooms = flowParams["rooms"].shape[1]
            flowParams["p_w"] = flowParams[pType]
            p0_meas = [row["mean-p-room"] for i in range(NRooms)]
            C_ds = getC(np.array(p0_meas), rho, flowParams)

            for i, windowKey in enumerate(windowKeys):
                flowStatsMI.loc[(run, windowKey), f"{pType}-C_d"] = C_ds[i]

            optResults = findOptimalP0(rho, flowParams)

            p0 = optResults.x
            roomVentilationMI.loc[(run, room), f"{pType}-p0"] = np.mean(p0)
            roomVentilationMI.loc[(run, room), f"{pType}-success"] = optResults.success
            qs = flowField(np.array(p0), rho, flowParams)

            for i, windowKey in enumerate(windowKeys):
                flowStatsMI.loc[(run, windowKey), f"{pType}-q_model"] = qs[i]
                flowStatsMI.loc[(run, windowKey), f"{pType}-netq_model"] = abs(qs[i])

            if "dual" in room and useDoors:
                qs = qs[0:-1]
            roomVentilationMI.loc[(run, room), f"{pType}-q_model"] = np.sum(abs(np.array(qs))) / 2

    return flowStatsMI, roomVentilationMI


In [ ]:
[1.16, 1.31, 1.15, 1, 0, 0]

flowStatsMI, roomVentilationMI = update_flow_and_ventilation(flowStatsMI, roomVentilationMI, [1, 1, 1, 1, 0, 0], useDoors=True,
    pTypes = {
        "pW": "mean-sn_prod(p)",
        "pEP": "EP_p_avg",
        "pNoInt": "pNoInt"
    }
)

In [ ]:
flowStatsMINoSL = flowStatsMI[flowStatsMI["AofA"] % 1 == 0].copy()
roomVentilationMINoSL = roomVentilationMI[roomVentilationMI["sinAofA"].notna()].copy()

In [ ]:
dfFit = flowStatsMI.dropna(subset=["pNoInt-q_model"])
# Reshape x into a 2D column vector
x = dfFit["pNoInt-q_model"].values.reshape(-1, 1)  # Ensure it's 2D
y = dfFit["mean-mass_flux"].values  # Keep y as 1D

# Solve for the slope (forcing intercept = 0)
CfitWindow, _, _, _ = np.linalg.lstsq(x, y, rcond=None)
CfitWindow = CfitWindow[0]

# Print the fitted coefficient
print("Fitted Coefficient:", CfitWindow)

# Fit C for each room type separately
room_types = dfFit['roomType'].unique()
CfitRoomTypes = {}

for room_type in room_types:
    dfRoomType = dfFit[dfFit['roomType'] == room_type]
    x = dfRoomType["pNoInt-q_model"].values.reshape(-1, 1)  # Ensure it's 2D
    y = dfRoomType["mean-mass_flux"].values  # Keep y as 1D

    # Solve for the slope (forcing intercept = 0)
    Cfit, _, _, _ = np.linalg.lstsq(x, y, rcond=None)
    CfitRoomTypes[room_type] = Cfit[0]

# Print the fitted coefficients for each room type
for room_type, Cfit in CfitRoomTypes.items():
    print(f"Fitted Coefficient for {room_type}: {Cfit}")



In [ ]:
# Create the scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=flowStatsMI, x='pNoInt-q_model', y="mean-mass_flux", hue="roomType", style="delT")

# Overlay the lines from the fitted C values
x_vals = np.linspace(flowStatsMI["mean-mass_flux"].min(), flowStatsMI["mean-mass_flux"].max(), 100)

# Overall fitted line
y_vals_overall = x_vals * CfitWindow
plt.plot(x_vals, y_vals_overall, label='Overall Fit', color='black')

# Fitted lines for each room type
for room_type, Cfit in CfitRoomTypes.items():
    y_vals_room = Cfit * x_vals
    plt.plot(x_vals, y_vals_room, label=f'{room_type} Fit')

plt.legend()
plt.xlabel('Mean Mass Flux')
plt.ylabel('pNoInt-q_model')
plt.title('Scatter Plot with Fitted Lines')

In [ ]:
# Create the scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=flowStatsMI, x="mean-mass_flux", y='pEP-q_model', hue="roomType", style="roomType")

# Overlay the lines from the fitted C values
x_vals = np.linspace(flowStatsMI["mean-mass_flux"].min(), flowStatsMI["mean-mass_flux"].max(), 100)


plt.legend()
plt.xlabel('Mean Mass Flux')
plt.ylabel('pEP-q_model')
plt.title('Scatter Plot with Fitted Lines')

In [ ]:
"""
Angles:
array([ 1.17340176,  1.32334274,  1.15833887,  0.00804852, -0.03391678,
        0.01388756])
0.14092224654844634

Shear/Normal:
array([ 1.16847614e+00,  1.31839869e+00,  1.17020841e+00,  1.62123154e-04,
        1.76884283e-04, -4.20435224e-04])
0.14068568319124428

Openning:
array([ 1.17383905e+00,  1.25949764e+00,  1.04268126e-05,  3.76369301e-05,
       -4.24760102e-05,  1.37200509e-04])
0.14497885547335324

AofA2:
array([ 1.17132613e+00,  1.31910337e+00,  2.03823868e-04, -4.39502373e-04,
        2.03392091e-04])
0.14071399814356256
"""

In [ ]:
# def qRMSE(df):
#     return np.sqrt(np.mean((df["pNoInt-q_model"] - df["mean-mass_flux"])**2))

# def objective(params, flowStatsMI = flowStatsMINoSL, roomVentilationMI = roomVentilationMINoSL):
#     display(params)
#     flowStatsMI, roomVentilationMI = update_flow_and_ventilation(flowStatsMI, roomVentilationMI, params, typeC_d="AofA3")
#     RMSE = qRMSE(roomVentilationMI)
#     display(RMSE)
#     return RMSE
# initial_guess = [ 1.17340176,  1.32334274,  1.15833887,  0.00804852, -0.03391678, 0.01388756, 0, 0]
# result = minimize(objective, initial_guess, method='Nelder-Mead')

In [ ]:
plotdf = roomVentilationMI.copy()
plotdf["x"] = plotdf["mean-p-room"]**0.5
plotdf["y"] = plotdf["pNoInt-p0"]**0.5
px.scatter(data_frame=plotdf, x="x", y='y', symbol = "roomType", color = "delT")

In [ ]:
px.scatter(data_frame=roomVentilationMI, x="pEP-p0", y='pNoInt-p0', symbol = "houseType", color = "roomType")

In [ ]:
px.box(data_frame=flowStatsMI.sort_values(["AofA", "roomType"]), x="pNoInt-C_d", color = "AofA", facet_col = "roomType")

In [ ]:
px.box(data_frame=flowStatsMI.sort_values(["AofA", "roomType"]), x="pNoInt-C_d", color = "AofA", facet_col = "roomType")

In [ ]:
px.box(data_frame=flowStatsMI.sort_values(["AofA", "roomType"]), x="pNoInt-C_d", color = "AofA", facet_col = "openingType")

In [ ]:
px.box(data_frame=flowStatsMI.sort_values(["AofA"]), x="pNoInt-C_d", color = "AofA")

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD

# Filter and sort the data
flowStatsMINoSL = flowStatsMI[flowStatsMI["AofA"] % 1 == 0]
flowStatsMINoSL.sort_values(by=["WS", "AofA"], inplace=True)

# Define substrings to keep and drop
subStrKeep = ["pNoInt-C_d", "sinAofA", "cosAofA", "roomType", "WS", "EP_normal", "EP_shear", "mean-mass_flux", "openingType"]
meanKmeansDf = flowStatsMINoSL[subStrKeep]

# Convert roomType to numeric
meanKmeansDf["roomType"] = meanKmeansDf["roomType"].astype('category').cat.codes
meanKmeansDf["openingType"] = meanKmeansDf["openingType"].astype('category').cat.codes

# Perform z-score normalization
mean = np.mean(meanKmeansDf, axis=0)
std = np.std(meanKmeansDf, axis=0)
meanKmeansDf = (meanKmeansDf - mean) / std

# Apply K-Means Algorithm
k = 4
kmeans = KMeans(n_clusters=k, random_state=0, n_init='auto')
kmeans.fit(meanKmeansDf)

# Assign cluster labels
flowStatsMINoSL["labels"] = kmeans.labels_.astype(str)

# Perform SVD decomposition
svd = TruncatedSVD(n_components=3)
meanKmeansDfLowDim = svd.fit_transform(meanKmeansDf)

# Visualize the clusters
fig = px.scatter_3d(data_frame=flowStatsMINoSL, x=meanKmeansDfLowDim[:, 0], y=meanKmeansDfLowDim[:, 1], z=meanKmeansDf["pNoInt-C_d"], color="labels")
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=1200, height=800, legend=dict(x=0, y=1.1))
fig.show()

In [ ]:
# Perform SVD decomposition
svd = TruncatedSVD(n_components=meanKmeansDf.shape[1])
meanKmeansDfLowDim = svd.fit_transform(meanKmeansDf)

# Plot the magnitude of the eigenvalues
plt.figure()
plt.plot(range(1, len(svd.explained_variance_) + 1), svd.explained_variance_, marker='o')
plt.xlabel('Component Number')
plt.ylabel('Eigenvalue Magnitude')
plt.title('Magnitude of Eigenvalues')
plt.grid(True)
plt.show()

In [ ]:
corrDf = meanKmeansDf.copy()
corrDf['roomType'] = flowStatsMINoSL['roomType']

# Split the data by roomType
room_types = corrDf['roomType'].unique()

# Plot correlation matrices for each roomType
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.flatten()

for ax, room_type in zip(axes, room_types):
    room_data = corrDf[corrDf['roomType'] == room_type]
    room_data = room_data.drop(columns='roomType')
    corr_matrix = room_data.corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.1f', linewidths=0.5, ax=ax)
    ax.set_title(f'Correlation Matrix for Room Type {room_type}')

plt.tight_layout()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Define the features (independent variables) and target (dependent variable)
features = ['AofA', 'roomType', 'WS', 'EP_normal', 'EP_shear', 'mean-mass_flux']
X = meanKmeansDf[features]
y = meanKmeansDf['pNoInt-C_d']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

# Display the coefficients
coefficients = pd.DataFrame(model.coef_, features, columns=['Coefficient'])
print(coefficients)

In [ ]:
px.scatter(data_frame=flowStatsMI, x="mean-mass_flux", y='pNoInt-q_model', color = "AofA_resid")

In [ ]:
import seaborn as sns

# Create the scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=flowStatsMI, x="mean-mass_flux", y='pEP-q_model', hue="roomType", style="roomType")

# Overlay the lines from the fitted C values
x_vals = np.linspace(flowStatsMI["mean-mass_flux"].min(), flowStatsMI["mean-mass_flux"].max(), 100)

# Overall fitted line
y_vals_overall = CfitWindow * x_vals
plt.plot(x_vals, y_vals_overall, label='Overall Fit', color='black')

# Fitted lines for each room type
for room_type, Cfit in CfitRoomTypes.items():
    y_vals_room = Cfit * x_vals
    plt.plot(x_vals, y_vals_room, label=f'{room_type} Fit')

plt.legend()
plt.xlabel('Mean Mass Flux')
plt.ylabel('pEP-q_model')
plt.title('Scatter Plot with Fitted Lines')
plt.show()

In [ ]:
px.scatter(data_frame=roomVentilationMI, x="mean-mass_flux", y='pNoInt-q_model', symbol = "houseType", color = "roomType")

In [ ]:
px.scatter(data_frame=roomVentilationMI, x="mean-mass_flux", y='pEP-q_model', symbol = "houseType", color = "roomType")

In [ ]:
roomVentilationMI["sinAofA"]

In [ ]:
px.scatter(data_frame=roomVentilationMI, x="mean-mass_flux", y='pEP-q_model', symbol = "houseType", color = "sinAofA")

In [ ]:
dfFit = roomVentilationMI.dropna(subset=["pNoInt-q_model"])
# Reshape x into a 2D column vector
x = dfFit["mean-mass_flux"].values.reshape(-1, 1)  # Ensure it's 2D
y = dfFit["pNoInt-q_model"].values  # Keep y as 1D

# Solve for the slope (forcing intercept = 0)
CfitRooms, _, _, _ = np.linalg.lstsq(x, y, rcond=None)
CfitRooms = CfitRooms[0]

# Print the fitted coefficient
print("Fitted Coefficient:", CfitRooms)

In [ ]:
CfitRooms = 1
roomVentilationMI["q_model-diff"] = roomVentilationMI["pNoInt-q_model"]/CfitRooms - roomVentilationMI["mean-mass_flux"]
roomVentilationMI["q_model-error"] = roomVentilationMI["q_model-diff"] / roomVentilationMI["mean-mass_flux"]

fig, axs = plt.subplots(2, 2, constrained_layout=True, figsize=(8, 6))
conditions = [(2, 0), (2, 45), (3, 0), (3, 45)]
for ax, (C, A) in zip(axs.flat, conditions):
    plotdf = roomVentilationMI[(roomVentilationMI["C"] == C) & (roomVentilationMI["A"] == A)]
    im = ax.scatter(plotdf['x'], plotdf['z'], s=8*im_scaling, c=plotdf["q_model-error"], cmap='PiYG', edgecolors="black", linewidths=.1, vmin=-2, vmax=2)
fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.02, pad=0.04)

fig, axs = plt.subplots(2, 2, constrained_layout=True, figsize=(8, 6))
for ax, (C, A) in zip(axs.flat, conditions):
    plotdf = roomVentilationMI[(roomVentilationMI["C"] == C) & (roomVentilationMI["A"] == A)]
    im = ax.scatter(plotdf['x'], plotdf['z'], s=8*im_scaling, c=plotdf["q_model-diff"], cmap='RdBu', edgecolors="black", linewidths=.1, vmin=-.25, vmax=.25)
fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.02, pad=0.04)

In [ ]:
flowStatsMI["q_model-diff"] = flowStatsMI["pNoInt-q_model"]/CfitRooms - flowStatsMI["mean-mass_flux"]
flowStatsMI["q_model-error"] = flowStatsMI["q_model-diff"] / flowStatsMI["mean-mass_flux"]

fig, axs = plt.subplots(2, 2, constrained_layout=True, figsize=(8, 6))
conditions = [(2, 0), (2, 45), (3, 0), (3, 45)]
for ax, (C, A) in zip(axs.flat, conditions):
    plotdf = flowStatsMI[(flowStatsMI["C"] == C) & (flowStatsMI["A"] == A)]
    im = ax.scatter(plotdf['x'], plotdf['z'], s=8*im_scaling, c=plotdf["q_model-error"], cmap='PiYG', edgecolors="black", linewidths=.1, vmin=-2, vmax=2)
fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.02, pad=0.04)

fig, axs = plt.subplots(2, 2, constrained_layout=True, figsize=(8, 6))
conditions = [(2, 0), (2, 45), (3, 0), (3, 45)]
for ax, (C, A) in zip(axs.flat, conditions):
    plotdf = flowStatsMI[(flowStatsMI["C"] == C) & (flowStatsMI["A"] == A)]
    im = ax.scatter(plotdf['x'], plotdf['z'], s=8*im_scaling, c=plotdf["q_model-diff"], cmap='RdBu', edgecolors="black", linewidths=.1, vmin=-.25, vmax=.25)
fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.02, pad=0.04)